## Bouw een logistiek regressiemodel - Les 4

![Infographic logistieke vs. lineaire regressie](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Pre-lecture quiz](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Introductie

In deze laatste les over regressie, een van de basis *klassieke* ML-technieken, gaan we kijken naar logistieke regressie. Je gebruikt deze techniek om patronen te ontdekken en binaire categorieën te voorspellen. Is dit snoep chocolade of niet? Is deze ziekte besmettelijk of niet? Zal deze klant dit product kiezen of niet?

In deze les leer je:

-   Technieken voor logistieke regressie

✅ Verdiep je kennis over het werken met dit type regressie in deze [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Vereisten

Nu we met de pompoendata hebben gewerkt, zijn we er inmiddels vertrouwd genoeg mee om te beseffen dat er één binaire categorie is waarmee we kunnen werken: `Color`.

Laten we een logistiek regressiemodel bouwen om te voorspellen, gegeven enkele variabelen, *welke kleur een bepaalde pompoen waarschijnlijk heeft* (oranje 🎃 of wit 👻).

> Waarom hebben we het over binaire classificatie in een lesgroep over regressie? Alleen om taalkundige redenen, aangezien logistieke regressie [eigenlijk een classificatiemethode](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) is, zij het een lineair gebaseerde. Leer meer over andere manieren om data te classificeren in de volgende lesgroep.

Voor deze les hebben we de volgende pakketten nodig:

-   `tidyverse`: Het [tidyverse](https://www.tidyverse.org/) is een [collectie van R-pakketten](https://www.tidyverse.org/packages) ontworpen om data science sneller, eenvoudiger en leuker te maken!

-   `tidymodels`: Het [tidymodels](https://www.tidymodels.org/) framework is een [collectie van pakketten](https://www.tidymodels.org/packages/) voor modelleren en machine learning.

-   `janitor`: Het [janitor-pakket](https://github.com/sfirke/janitor) biedt eenvoudige tools voor het onderzoeken en opschonen van rommelige data.

-   `ggbeeswarm`: Het [ggbeeswarm-pakket](https://github.com/eclarke/ggbeeswarm) biedt methoden om beeswarm-stijl grafieken te maken met ggplot2.

Je kunt ze installeren met:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternatief controleert het onderstaande script of je de benodigde pakketten hebt om deze module te voltooien en installeert ze voor je als ze ontbreken.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Definieer de vraag**

Voor onze doeleinden zullen we dit uitdrukken als een binair: 'Wit' of 'Niet Wit'. Er is ook een 'gestreepte' categorie in onze dataset, maar er zijn weinig gevallen van, dus we zullen deze niet gebruiken. Het verdwijnt sowieso zodra we de null-waarden uit de dataset verwijderen.

> 🎃 Leuk weetje: we noemen witte pompoenen soms 'spookpompoenen'. Ze zijn niet zo makkelijk te snijden, dus ze zijn niet zo populair als de oranje, maar ze zien er wel gaaf uit! We zouden onze vraag dus ook kunnen herformuleren als: 'Spook' of 'Niet Spook'. 👻

## **Over logistische regressie**

Logistische regressie verschilt op een aantal belangrijke manieren van lineaire regressie, waar je eerder over hebt geleerd.

#### **Binaire classificatie**

Logistische regressie biedt niet dezelfde mogelijkheden als lineaire regressie. De eerste biedt een voorspelling over een `binaire categorie` ("oranje of niet oranje"), terwijl de laatste in staat is om `continue waarden` te voorspellen, bijvoorbeeld gegeven de herkomst van een pompoen en het tijdstip van oogsten, *hoeveel de prijs zal stijgen*.

![Infographic door Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Andere classificaties

Er zijn andere soorten logistische regressie, waaronder multinomiaal en ordinaal:

- **Multinomiaal**, waarbij er meer dan één categorie is - "Oranje, Wit en Gestreept".

- **Ordinaal**, waarbij er geordende categorieën zijn, handig als we onze uitkomsten logisch willen ordenen, zoals onze pompoenen die geordend zijn op een eindig aantal maten (mini,sm,med,lg,xl,xxl).

![Multinomiaal vs ordinale regressie](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Variabelen HOEVEN NIET te correleren**

Weet je nog dat lineaire regressie beter werkte met meer gecorreleerde variabelen? Logistische regressie is het tegenovergestelde - de variabelen hoeven niet te correleren. Dat werkt voor deze data, die enigszins zwakke correlaties heeft.

#### **Je hebt veel schone data nodig**

Logistische regressie geeft nauwkeurigere resultaten als je meer data gebruikt; onze kleine dataset is niet optimaal voor deze taak, dus houd dat in gedachten.

✅ Denk na over de soorten data die zich goed lenen voor logistische regressie.

## Oefening - maak de data schoon

Maak eerst de data een beetje schoon door null-waarden te verwijderen en alleen enkele kolommen te selecteren:

1. Voeg de volgende code toe:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Je kunt altijd een kijkje nemen in je nieuwe dataframe door de functie [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) te gebruiken, zoals hieronder:


In [ ]:
pumpkins_select %>% 
  glimpse()


Laten we bevestigen dat we daadwerkelijk een binair classificatieprobleem gaan aanpakken:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Visualisatie - categorische plot
Je hebt inmiddels de pompoendata opnieuw geladen en schoongemaakt, zodat je een dataset hebt met een paar variabelen, waaronder Kleur. Laten we de dataframe in het notebook visualiseren met behulp van de ggplot-bibliotheek.

De ggplot-bibliotheek biedt handige manieren om je data te visualiseren. Bijvoorbeeld, je kunt de verdelingen van de data vergelijken voor elke Variëteit en Kleur in een categorische plot.

1. Maak zo'n plot door de geombar-functie te gebruiken, met onze pompoendata, en specificeer een kleurmapping voor elke pompoencategorie (oranje of wit):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Door de gegevens te bekijken, kun je zien hoe de Kleuren-gegevens zich verhouden tot de Variëteit.

✅ Gezien deze categorische plot, welke interessante onderzoeken kun je bedenken?


### Voorbewerking van data: feature encoding

Onze dataset met pompoenen bevat stringwaarden voor alle kolommen. Werken met categorische data is intuïtief voor mensen, maar niet voor machines. Machine learning-algoritmes werken goed met getallen. Daarom is encoding een zeer belangrijke stap in de fase van data-voorbewerking, omdat het ons in staat stelt om categorische data om te zetten in numerieke data, zonder informatie te verliezen. Goede encoding leidt tot het bouwen van een goed model.

Voor feature encoding zijn er twee hoofdtypen encoders:

1. Ordinale encoder: dit is geschikt voor ordinale variabelen, wat categorische variabelen zijn waarbij de data een logische volgorde volgt, zoals de kolom `item_size` in onze dataset. Het maakt een mapping waarbij elke categorie wordt weergegeven door een getal, dat overeenkomt met de volgorde van de categorie in de kolom.

2. Categorische encoder: dit is geschikt voor nominale variabelen, wat categorische variabelen zijn waarbij de data geen logische volgorde volgt, zoals alle kenmerken die anders zijn dan `item_size` in onze dataset. Dit is een one-hot encoding, wat betekent dat elke categorie wordt weergegeven door een binaire kolom: de gecodeerde variabele is gelijk aan 1 als de pompoen tot die Variety behoort en anders 0.

Tidymodels biedt nog een ander handig pakket: [recipes](https://recipes.tidymodels.org/) - een pakket voor het voorbewerken van data. We definiëren een `recipe` die aangeeft dat alle voorspellerkolommen moeten worden gecodeerd in een reeks gehele getallen, `prep` om de benodigde hoeveelheden en statistieken te schatten die nodig zijn voor bewerkingen, en uiteindelijk `bake` om de berekeningen toe te passen op nieuwe data.

> Normaal gesproken wordt recipes meestal gebruikt als een voorbewerker voor modellering, waarbij het definieert welke stappen moeten worden toegepast op een dataset om deze klaar te maken voor modellering. In dat geval wordt het **ten zeerste aanbevolen** om een `workflow()` te gebruiken in plaats van handmatig een recipe te schatten met prep en bake. We zullen dit allemaal zo meteen zien.
>
> Voor nu gebruiken we echter recipes + prep + bake om te specificeren welke stappen moeten worden toegepast op een dataset om deze klaar te maken voor data-analyse en vervolgens de voorbewerkte data met de toegepaste stappen te extraheren.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Wat zijn de voordelen van het gebruik van een ordinale encoder voor de kolom Item Size?

### Analyseer relaties tussen variabelen

Nu we onze gegevens hebben voorbewerkt, kunnen we de relaties tussen de kenmerken en het label analyseren om een idee te krijgen van hoe goed het model in staat zal zijn om het label te voorspellen op basis van de kenmerken. De beste manier om dit soort analyse uit te voeren is door de gegevens te visualiseren.  
We zullen opnieuw de ggplot-functie geom_boxplot_ gebruiken om de relaties tussen Item Size, Variety en Color in een categorische plot te visualiseren. Om de gegevens beter te plotten, gebruiken we de gecodeerde kolom Item Size en de niet-gecodeerde kolom Variety.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Gebruik een zwermplot

Omdat Kleur een binaire categorie is (Wit of Niet), heeft het 'een [gespecialiseerde aanpak](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) nodig voor visualisatie'.

Probeer een `zwermplot` om de verdeling van kleur ten opzichte van de item_grootte te laten zien.

We gebruiken het [ggbeeswarm-pakket](https://github.com/eclarke/ggbeeswarm), dat methoden biedt om zwermachtige plots te maken met ggplot2. Zwermplots zijn een manier om punten te plotten die normaal gesproken overlappen, zodat ze naast elkaar worden geplaatst.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Nu we een idee hebben van de relatie tussen de binaire categorieën van kleur en de grotere groep van maten, laten we logistieke regressie verkennen om de waarschijnlijke kleur van een bepaalde pompoen te bepalen.

## Bouw je model

Selecteer de variabelen die je wilt gebruiken in je classificatiemodel en splits de data in trainings- en testsets. [rsample](https://rsample.tidymodels.org/), een pakket in Tidymodels, biedt infrastructuur voor efficiënte datasplitsing en herhaald steekproefnemen:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 We zijn nu klaar om een model te trainen door de trainingskenmerken aan het trainingslabel (kleur) te koppelen.

We beginnen met het maken van een recept dat de preprocessiestappen specificeert die op onze gegevens moeten worden uitgevoerd om ze klaar te maken voor modellering, bijvoorbeeld: het coderen van categorische variabelen naar een reeks gehele getallen. Net zoals `baked_pumpkins` maken we een `pumpkins_recipe`, maar we gebruiken geen `prep` en `bake`, omdat dit wordt opgenomen in een workflow, zoals je over een paar stappen zult zien.

Er zijn behoorlijk wat manieren om een logistisch regressiemodel te specificeren in Tidymodels. Zie `?logistic_reg()`. Voor nu specificeren we een logistisch regressiemodel via de standaard `stats::glm()` engine.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Nu we een recept en een modelspecificatie hebben, moeten we een manier vinden om deze samen te bundelen in een object dat eerst de gegevens zal voorbewerken (prep+bake achter de schermen), het model zal trainen op de voorbewerkte gegevens en ook ruimte biedt voor eventuele nabewerkingsactiviteiten.

In Tidymodels wordt dit handige object een [`workflow`](https://workflows.tidymodels.org/) genoemd en het bevat op een handige manier je modelcomponenten.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Nadat een workflow is *gespecificeerd*, kan een model worden `getraind` met behulp van de [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) functie. De workflow zal een recept inschatten en de gegevens vooraf verwerken voordat het trainen begint, zodat we dat niet handmatig hoeven te doen met prep en bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Het model toont de coëfficiënten die tijdens de training zijn geleerd.

Nu we het model hebben getraind met de trainingsdata, kunnen we voorspellingen doen op de testdata met behulp van [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Laten we beginnen met het gebruiken van het model om labels voor onze testset te voorspellen en de waarschijnlijkheden voor elk label te berekenen. Wanneer de waarschijnlijkheid groter is dan 0,5, is de voorspelde klasse `WHITE`, anders `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Heel goed! Dit biedt wat meer inzicht in hoe logistische regressie werkt.

### Betere begrip via een confusion matrix

Het vergelijken van elke voorspelling met de bijbehorende "ground truth" werkelijke waarde is niet de meest efficiënte manier om te bepalen hoe goed het model voorspelt. Gelukkig heeft Tidymodels nog een paar handige tools: [`yardstick`](https://yardstick.tidymodels.org/) - een pakket dat wordt gebruikt om de effectiviteit van modellen te meten met behulp van prestatie-indicatoren.

Een prestatie-indicator die vaak wordt gebruikt bij classificatieproblemen is de [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix). Een confusion matrix beschrijft hoe goed een classificatiemodel presteert. Het geeft weer hoeveel voorbeelden in elke klasse correct zijn geclassificeerd door een model. In ons geval laat het zien hoeveel oranje pompoenen correct als oranje zijn geclassificeerd en hoeveel witte pompoenen correct als wit zijn geclassificeerd; de confusion matrix laat ook zien hoeveel voorbeelden in de **verkeerde** categorieën zijn ingedeeld.

De functie [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) van yardstick berekent deze kruistabel van waargenomen en voorspelde klassen.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Laten we de verwarringsmatrix interpreteren. Ons model wordt gevraagd om pompoenen te classificeren in twee binaire categorieën: categorie `wit` en categorie `niet-wit`.

-   Als je model een pompoen voorspelt als wit en deze behoort in werkelijkheid tot de categorie 'wit', noemen we dit een `true positive`, weergegeven door het getal linksboven.

-   Als je model een pompoen voorspelt als niet-wit en deze behoort in werkelijkheid tot de categorie 'wit', noemen we dit een `false negative`, weergegeven door het getal linksonder.

-   Als je model een pompoen voorspelt als wit en deze behoort in werkelijkheid tot de categorie 'niet-wit', noemen we dit een `false positive`, weergegeven door het getal rechtsboven.

-   Als je model een pompoen voorspelt als niet-wit en deze behoort in werkelijkheid tot de categorie 'niet-wit', noemen we dit een `true negative`, weergegeven door het getal rechtsonder.

| Waarheid |
|:--------:|


|               |        |       |
|---------------|--------|-------|
| **Voorspeld** | WIT    | ORANJE |
| WIT           | TP     | FP    |
| ORANJE        | FN     | TN    |

Zoals je misschien al vermoedt, is het wenselijk om een groter aantal true positives en true negatives te hebben, en een lager aantal false positives en false negatives. Dit impliceert dat het model beter presteert.

De verwarringsmatrix is nuttig omdat deze leidt tot andere statistieken die ons kunnen helpen de prestaties van een classificatiemodel beter te evalueren. Laten we enkele van deze statistieken doornemen:

🎓 Precisie: `TP/(TP + FP)` gedefinieerd als het aandeel voorspelde positieven dat daadwerkelijk positief is. Ook wel [positieve voorspellende waarde](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value") genoemd.

🎓 Recall: `TP/(TP + FN)` gedefinieerd als het aandeel positieve resultaten uit het aantal monsters dat daadwerkelijk positief was. Ook bekend als `gevoeligheid`.

🎓 Specificiteit: `TN/(TN + FP)` gedefinieerd als het aandeel negatieve resultaten uit het aantal monsters dat daadwerkelijk negatief was.

🎓 Nauwkeurigheid: `TP + TN/(TP + TN + FP + FN)` Het percentage labels dat correct is voorspeld voor een monster.

🎓 F-Score: Een gewogen gemiddelde van de precisie en recall, waarbij 1 het beste is en 0 het slechtste.

Laten we deze statistieken berekenen!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Visualiseer de ROC-curve van dit model

Laten we nog een visualisatie maken om de zogenaamde [`ROC-curve`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) te bekijken:


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC-curves worden vaak gebruikt om een beeld te krijgen van de prestaties van een classifier in termen van echte versus valse positieven. ROC-curves tonen meestal de `True Positive Rate`/Sensitiviteit op de Y-as en de `False Positive Rate`/1-Specificiteit op de X-as. De steilheid van de curve en de ruimte tussen de middellijn en de curve zijn belangrijk: je wilt een curve die snel omhoog gaat en de lijn overschrijdt. In ons geval zijn er in het begin valse positieven, waarna de lijn correct omhoog en over de lijn gaat.

Laten we tot slot `yardstick::roc_auc()` gebruiken om de daadwerkelijke Area Under the Curve te berekenen. Een manier om AUC te interpreteren is als de kans dat het model een willekeurig positief voorbeeld hoger rangschikt dan een willekeurig negatief voorbeeld.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Het resultaat is ongeveer `0.975`. Aangezien de AUC varieert van 0 tot 1, wil je een hoge score, omdat een model dat 100% correct is in zijn voorspellingen een AUC van 1 zal hebben; in dit geval is het model *best goed*.

In toekomstige lessen over classificaties leer je hoe je de scores van je model kunt verbeteren (zoals het omgaan met onevenwichtige data in dit geval).

## 🚀Uitdaging

Er is nog veel meer te ontdekken over logistische regressie! Maar de beste manier om te leren is door te experimenteren. Zoek een dataset die zich leent voor dit type analyse en bouw er een model mee. Wat leer je? tip: probeer [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) voor interessante datasets.

## Review & Zelfstudie

Lees de eerste paar pagina's van [dit artikel van Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) over enkele praktische toepassingen van logistische regressie. Denk na over taken die beter geschikt zijn voor het ene of het andere type regressietaken die we tot nu toe hebben bestudeerd. Wat zou het beste werken?



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, willen we u erop wijzen dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
